# Vision Transformer를 활용한 식물 질환 분류

- [관련 논문]

    https://arxiv.org/pdf/2010.11929.pdf (ViT 제안)

    https://www.frontiersin.org/articles/10.3389/fpls.2016.01419/full (간접 관련)
    
    https://www.sciencedirect.com/science/article/pii/S2666285X22000218 (간접 관련)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모델 준비

## 필요 라이브러리 설치

In [ ]:
!pip install datasets
!pip install transformers
!pip install torchmetrics
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.1
    Uninstalling huggingface-hub-0.19.1:
      Successfully uninstalled huggingface-hub-0.19.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.4 MB/s eta 0

## 데이터셋 다운로드



- 총 38개의 카테고리가 있음. 모든 데이터의 비율이 균등하지 않다는 점에 유의

- [데이터셋]

- [관련 논문] Hughes, David P., and Marcel Salathe. “An Open Access Repository of Images on Plant Health to Enable the Development of Mobile Disease Diagnostics.” ArXiv:1511.08060 [Cs], Apr. 2016. arXiv.org, http://arxiv.org/abs/1511.08060.

- [출처] https://github.com/spMohanty/PlantVillage-Dataset

In [ ]:
import os
import io
import shutil
import requests
from zipfile import ZipFile

def generate_dataset_dir(data_dir, test_rate=0.2, valid_rate=0.2, gap=1):
    """
    made by hope04302
    zip파일의 데이터셋을 train, validation, test로 분리
    """

    zip_path = "https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"
    zip = requests.get(zip_path)

    if os.path.isdir(data_dir):
        shutil.rmtree(data_dir)

    with ZipFile(io.BytesIO(zip.content), 'r') as zf:
        zf.extractall(f"{data_dir}")

    in_path = f"{data_dir}/Plant_leave_diseases_dataset_without_augmentation"
    clss = os.listdir(in_path)

    for cls in clss:

        for folder in ["train", "validation", "test"]:
            os.makedirs(f"{data_dir}/{folder}/{cls}")

        images = os.listdir(f"{in_path}/{cls}")

        train_idx = int(len(images) * (1 - test_rate) * (1 - valid_rate))
        valid_idx = int(len(images) * (1 - test_rate))

        for image in images[:train_idx:gap]:
            if image[-3:].lower() == "jpg":
                shutil.move(f"{in_path}/{cls}/{image}", f"{data_dir}/train/{cls}")

        for image in images[train_idx:valid_idx:gap]:
            if image[-3:].lower() == "jpg":
                shutil.move(f"{in_path}/{cls}/{image}", f"{data_dir}/validation/{cls}")

        for image in images[valid_idx::gap]:
            if image[-3:].lower() == "jpg":
                shutil.move(f"{in_path}/{cls}/{image}", f"{data_dir}/test/{cls}")

    shutil.rmtree(in_path)

In [ ]:
from datasets import load_dataset

generate_dataset_dir(data_dir='/content/plantvillage', test_rate=0.95, gap=1)
dataset = load_dataset("imagefolder", data_dir='/content/plantvillage')

Resolving data files:   0%|          | 0/2203 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/552 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52693 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 모델, 토크나이저 다운로드(중요)

In [ ]:
# 실제 실험에서는 다르게 설정할 것.
# NUM_LABELS는 건들지 말 것.

NUM_LABELS = 39
BATCH_SIZE = 32
EPOCHS = 200

In [ ]:
import torch
from transformers import AutoImageProcessor

# 수정 1 =====================================

# model_ckpt: 원하는 모델을 아래에서 검색 후, 이름 붙여넣기(ctrl + click하면 들어가짐)
# https://huggingface.co/models?pipeline_tag=image-classification&sort=trending

# model_name: 모델을 드라이브에 저장할 이름
# save_dir: 모델을 저장할 경로

model_ckpt = "google/bit-50"
save_dir = f'/content/drive/MyDrive/saki_model/{model_ckpt}'

# ============================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

processor = AutoImageProcessor.from_pretrained(model_ckpt)

In [ ]:
from transformers import AutoModelForImageClassification

# 수정 2-1 =====================================

# [주의] 수정 2-1, 2-2 중 하나는 주석 처리할 것.
# 만약 원래 코드를 그대로 쓰고 싶으면 선택

model = (AutoModelForImageClassification
         .from_pretrained(model_ckpt, num_labels=NUM_LABELS, ignore_mismatched_sizes=True)
         .to(device))

# ==============================================

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# import torch.nn as nn
# from transformers import AutoModel

# # 수정 2-2 =====================================

# # [주의] 수정 2-1, 2-2 중 하나는 주석 처리할 것.
# # 원래 코드의 classifier를 고치고 싶다면 선택

# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.model = AutoModel.from_pretrained(model_ckpt)
#         self.classifier = nn.Linear(self.model.config.hidden_size, NUM_LABELS)
#         self.loss = nn.BCEWithLogitsLoss()

#     def forward(self, pixel_values, labels=None):

#         # hidden.last_hidden_state도 있음...

#         hidden = self.model(pixel_values=pixel_values).pooler_output
#         logits = self.classifier(hidden)
#         loss = self.loss(logits, labels)

#         model_output = {
#             'loss': loss,
#             'logits': logits
#         }

#         return model_output

# model = Model().to(device)

# # ==============================================

## 데이터셋, 모델 상태 확인

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 2203
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 552
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 52693
    })
})

In [ ]:
dataset["train"].features["label"].names

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Background_without_leaves',
 'Blueberry___healthy',
 'Cherry___Powdery_mildew',
 'Cherry___healthy',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn___Common_rust',
 'Corn___Northern_Leaf_Blight',
 'Corn___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot'

In [ ]:
processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [ ]:
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

## 모델 학습 준비

In [ ]:
import torch.nn.functional as F

def transform(example_batch):
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.stack(
            [F.one_hot(torch.tensor(x['labels']), num_classes=NUM_LABELS).type(torch.float) for x in batch]
        )
    }

In [ ]:
import torch
import torch.nn.functional as F
from torchmetrics import Accuracy, F1Score, HammingDistance, AUROC, ExactMatch

def compute_metrics(eval_pred):


    # 수정 3 ==============================================

    # eval_pred: model에서 나온 output값.
    # 여기를 수정하면 원하는 방향으로 metrics를 조작 가능

    logits, labels = eval_pred
    logits = torch.Tensor(logits)
    labels = torch.Tensor(labels).long()

    probs = logits
    preds = torch.zeros(size = probs.size())
    for i in range(len(probs)):
        preds[i][torch.argmax(probs[i])] = 1

    accuracy = Accuracy(task='multilabel', num_labels=NUM_LABELS)
    f1_macro = F1Score(task="multilabel", num_labels=NUM_LABELS, average='macro')
    f1_micro = F1Score(task="multilabel", num_labels=NUM_LABELS, average='micro')
    f1_weight = F1Score(task="multilabel", num_labels=NUM_LABELS, average='weighted')
    em = ExactMatch(task='multiclass', num_classes=2)
    auroc = AUROC(task='multilabel', num_labels=NUM_LABELS, average='micro')
    hamming = HammingDistance(task="multiclass", num_classes=2)

    each_f1_score = F1Score(task='multilabel', num_labels=NUM_LABELS, average=None)

    metrics = {'accuracy': accuracy(preds, labels),
               'f1_macro': f1_macro(preds, labels),
               'f1_micro': f1_micro(preds, labels),
               'f1_weighted': f1_weight(preds, labels),
               'auroc': auroc(preds, labels),
               'em': em(preds, labels),
               'hamming_loss': hamming(preds, labels)} | dict(zip(map(str, range(NUM_LABELS)), each_f1_score(preds, labels)))

    # =======================================================

    return metrics

In [ ]:
train, validation, test = dataset.with_transform(transform).values()

# 모델 학습

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/model",
    do_train=True,
    do_eval=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    run_name="v1",
    seed=42,
    remove_unused_columns=False,

    # 수정 4 =============================

    # 부정확하지만 빠르고 용량 적게 먹는 학습:
    # evaluation_strategy="epoch", save_strategy="no"

    # 좀 더 정확한 학습:
    evaluation_strategy="epoch", save_strategy="epoch", load_best_model_at_end=True, metric_for_best_model='eval_f1_macro', save_total_limit=1,

    # 좀 더 많이 정확한 학습(숫자 수정 가능):
    #evaluation_strategy="steps", eval_steps=100, save_strategy="steps", load_best_model_at_end=True, metric_for_best_model='eval_f1_score', save_total_limit=1,

    # ====================================

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=validation,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    data_collator=collate_fn,
)

In [ ]:
try: trainer.train()
finally: trainer.save_model()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

# 모델 평가

In [ ]:
training_args = TrainingArguments(
    output_dir=save_dir,
    do_train=True,
    do_eval=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    run_name="v1",
    seed=42,
    remove_unused_columns=False,
)

model = AutoModelForImageClassification.from_pretrained("/content/model").to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    data_collator=collate_fn,
)

In [ ]:
try: print(trainer.evaluate())
finally: trainer.save_model()

In [ ]:
import pandas as pd
from torch import tensor

a = {'eval_loss': 0.007293897680938244, 'eval_accuracy': 0.9979521036148071, 'eval_f1_score': 0.9610897898674011, 'eval_auroc': 0.9715272784233093, 'eval_recall': 0.9610897898674011, 'eval_precision': 0.9610897898674011, 'eval_0': 0.9348914623260498, 'eval_1': 0.9439002871513367, 'eval_2': 0.978723406791687, 'eval_3': 0.9713168144226074, 'eval_4': 0.9909090995788574, 'eval_5': 0.9720101952552795, 'eval_6': 0.968137264251709, 'eval_7': 0.8056460618972778, 'eval_8': 0.9977924823760986, 'eval_9': 0.9023883938789368, 'eval_10': 0.998643159866333, 'eval_11': 0.9737556576728821, 'eval_12': 0.9800676703453064, 'eval_13': 0.9874030947685242, 'eval_14': 0.966625452041626, 'eval_15': 0.9981871843338013, 'eval_16': 0.9843288660049438, 'eval_17': 0.9479768872261047, 'eval_18': 0.9586426019668579, 'eval_19': 0.9671787619590759, 'eval_20': 0.9717513918876648, 'eval_21': 0.9022801518440247, 'eval_22': 0.8054607510566711, 'eval_23': 0.9841726422309875, 'eval_24': 0.9904054403305054, 'eval_25': 0.9991401433944702, 'eval_26': 0.9816778898239136, 'eval_27': 0.9838337302207947, 'eval_28': 0.9446237683296204, 'eval_29': 0.7343904972076416, 'eval_30': 0.8848314881324768, 'eval_31': 0.9044075608253479, 'eval_32': 0.9190462231636047, 'eval_33': 0.9388272762298584, 'eval_34': 0.864098846912384, 'eval_35': 0.9904612302780151, 'eval_36': 0.9533527493476868, 'eval_37': 0.9835957884788513, 'eval_runtime': 840.3154, 'eval_samples_per_second': 61.425, 'eval_steps_per_second': 1.92}
a  = pd.DataFrame(a.values(), index=a.keys())
a